In [1]:
import json
import requests
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By

In [61]:
API_Key = None # str

In [2]:
driver = webdriver.Chrome()
driver.maximize_window()

In [74]:
def get_longitude_and_latitude(Station: str, API_Key:str):
    address = "捷運" + Station + "站"
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_Key}&language=zh"
    req = requests.get(url)
    geo_info = json.loads(req.text)
    if geo_info['status'] == 'ZERO_RESULTS':
        return [address, None, None]
    else:
        address = geo_info['results'][0]['formatted_address']
        lat_lng =  geo_info['results'][0]['geometry']['location']
        return [address, lat_lng['lat'], lat_lng['lng']]

In [75]:
# Taipei
temp_list = []
url = "https://web.metro.taipei/pages/tw/ticketroutetimesingle/071"
driver.get(url)
sleep(3)
element_table = driver.find_element(by=By.TAG_NAME, value="table").find_elements(by=By.TAG_NAME, value="tr")
StationNames = [i.text.split()[4] for i in element_table[2:]]
for Station in StationNames:
    temp_list.append(get_longitude_and_latitude(Station, API_Key))

In [78]:
df = pd.DataFrame(temp_list, columns=['地址', '緯度', '經度'])
df.to_csv("./metro_info/Taipei.csv")

In [35]:
# Taoyuan
url = "https://www.tymetro.com.tw/tymetro-new/tw/_pages/travel-guide/timetable-search.php"
driver.get(url)
StationNames = driver.find_element(by=By.ID, value="start_station").text.split("\n")[1:-1]

In [48]:
# Taichung
url = "https://www.tmrt.com.tw/metro-life/ride-time-and-fare"
driver.get(url)
StationNames = driver.find_element(by=By.NAME, value="stationSelection").text.split("\n")[1:-1]

In [56]:
# Kaohsiung
url = "https://www.krtc.com.tw/"
driver.get(url)
StationNames = driver.find_element(by=By.ID, value="KRTCStartStation").text.split("\n")[1:-1]
StationNames = [i.strip() for i in StationNames]

In [79]:
driver.close()